In [1]:
import torch
import scvi
import scgen
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
Global seed set to 0
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [2]:
# from gradients import run_expected_jacobian_scanvi, run_integrated_jacobian_scanvi

def batch_to_dict_scanvi(batch):
    return dict(x=batch["X"], batch_index=batch["batch"])

def batch_jacobian(outpt, inpt):
    n_out = outpt.shape[-1]
    
    jacs = []
    
    ones = torch.ones(outpt.shape[0]).to(outpt.device)
    
    for i in range(n_out):
        retain_graph = i!= n_out-1
        jacs.append(torch.autograd.grad(outpt[..., i], inpt, ones, retain_graph=retain_graph)[0])

    return torch.stack(jacs, dim=-1)

def expected_jacobian_step(func, inpt_dict, backprop_inpt_key, prime_inpt):
    x = inpt_dict[backprop_inpt_key]
    x_prime = prime_inpt.to(x.device)
    
    unif_coef = torch.rand(x.shape[0])[:, None].to(x.device)
    
    x_diff = x - x_prime
    
    new_inpt_dict = dict(inpt_dict)
    new_inpt_dict[backprop_inpt_key] = x_prime + x_diff * unif_coef

#     out = func(**new_inpt_dict)
    out = func(new_inpt_dict)  # edited bc I can't figure out how otherwise
    
    jac = batch_jacobian(out, x)
    
    return jac * x_diff[..., None].detach()

def integrated_jacobian(func, inpt_dict, backprop_inpt_key, prime_inpt=None, n_steps=10):
    x = inpt_dict[backprop_inpt_key]
    if prime_inpt is not None:
        x_prime = prime_inpt.to(x.device)
    else:
        x_prime = torch.zeros_like(x)
    
    x_diff = x - x_prime
    
    jacs = []
    
    new_inpt_dict = dict(inpt_dict)
    
    for i in range(n_steps):
        new_inpt_dict[backprop_inpt_key] = x_prime + x_diff * (i+1)/n_steps
        
        out = func(new_inpt_dict)  # edited
        
        jacs.append(batch_jacobian(out, x))
    
    return sum(jacs) * (1/n_steps) * x_diff[..., None].detach()

def run_expected_jacobian_scanvi(module_func, dl_base, dl_prime, n_steps=10, apply_abs=False, sum_obs=False):
    expected_jacs = []

    for batch in dl_base:
        inpt_dict = batch_to_dict_scanvi(batch)
        inpt_dict["x"].requires_grad = True
        inpt_dict["x"] = inpt_dict["x"].cuda()

        inpt_batch_size = inpt_dict["x"].shape[0]

        jacs_batch = []
        for i, batch_prime in enumerate(dl_prime):
            if i > n_steps-1:
                break

            prime_inpt = batch_to_dict_scanvi(batch_prime)["x"]
            if inpt_batch_size < prime_inpt.shape[0]:
                prime_inpt = prime_inpt[:inpt_batch_size]

            jacs_batch.append(expected_jacobian_step(module_func, inpt_dict, "x", prime_inpt))

        exp_jac_batch = sum(jacs_batch) / len(jacs_batch)
        if apply_abs:
            exp_jac_batch = exp_jac_batch.abs()
        
        expected_jacs.append(exp_jac_batch.cpu() if not sum_obs else exp_jac_batch.sum(0).cpu())
        
    if sum_obs:
        result = torch.stack(expected_jacs).sum(0)
    else:
        result = torch.cat(expected_jacs)
    return result

def run_integrated_jacobian_scanvi(module_func, dl_base, n_steps=10, apply_abs=False, sum_obs=False):
    integrated_jacs = []

    for batch in dl_base:
        inpt_dict = batch_to_dict_scanvi(batch)
        inpt_dict["x"].requires_grad = True
        inpt_dict["x"] = inpt_dict["x"].cuda()
        
        integr_jac_batch = integrated_jacobian(module_func, inpt_dict, "x", n_steps=n_steps)
        if apply_abs:
            integr_jac_batch = integr_jac_batch.abs()

        integrated_jacs.append(integr_jac_batch.cpu() if not sum_obs else integr_jac_batch.sum(0).cpu())
        
    if sum_obs:
        result = torch.stack(integrated_jacs).sum(0)
    else:
        result = torch.cat(integrated_jacs)
    return result

In [40]:
base_path = '/home/icb/yuge.ji/projects/feature-attribution-sc'
adata = sc.read(f'{base_path}/datasets/scgen_norman19.h5ad')
# adata = adata[~adata.obs.perturbation_name.isin(['C19orf26', 'C3orf72', 'KIAA1804'])].copy()  # do not match to gene, remove
adata

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/guide_identity' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/guide_identity' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/read_count' was written without encoding metadata.
  return read_elem(dataset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/obsm' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/obsm/X_pca' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/obsm/X_umap' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/obsp' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/uns' wa

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/varm' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/varm/PCs' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}


AnnData object with n_obs × n_vars = 69570 × 19018
    obs: 'guide_identity', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'guide_AHR', 'guide_ARID1A', 'guide_ARRDC3', 'guide_ATL1', 'guide_BAK1', 'guide_BCL2L11', 'guide_BCORL1', 'guide_BPGM', 'guide_C19orf26', 'guide_C3orf72', 'guide_CBFA2T3', 'guide_CBL', 'guide_CDKN1A', 'guide_CDKN1B', 'guide_CDKN1C', 'guide_CEBPA', 'guide_CEBPB', 'guide_CEBPE', 'guide_CELF2', 'guide_CITED1', 'guide_CKS1B', 'guide_CLDN6', 'guide_CNN1', 'guide_CNNM4', 'guide_COL1A1', 'guide_COL2A1', 'guide_CSRNP1', 'guide_DLX2', 'guide_DUSP9', 'guide_EGR1', 'guide_ELMSAN1', 'guide_ETS2', 'guide_FEV', 'guide_FOSB', 'guide_FOXA1', 'guide_FOXA3', 'guide_FOXF1', 'guide_FOXL2', 'guide_FOXO4', 'guide_GLB1L2', 'guide_HES7', 'guide_HK2', 'guide_HNF4A', 'guide_HOXA13', 'guide_HOXB9', 'guide_HOXC13', 'guide_IER5L', 'guide_IGDCC3', 'guide_IKZF3', 'guide_IRF1', 'guide_ISL2', 'guide_JUN', 'guide_KIAA1804', 'guide_KIF18B', 'guide_KIF2C', 'g

In [41]:
file = 'scgen_norman19_model1_shuffled'
model = scgen.SCGEN.load(f'{base_path}/models/{file}', adata=adata)
model

INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model1_
         shuffled/model.pt already downloaded                                                


SCGEN Model with the following params: 
n_hidden: 800, n_latent: 100, n_layers: 2, dropout_rate: 0.2
Training status: Trained

In [42]:
forw = lambda inpt_dict: model.module.inference(inpt_dict['x'])['z']

In [43]:
adata.obs["perturbation_name"].value_counts()

control    11835
KLF1        1954
BAK1        1451
CEBPE       1230
ETS2        1201
           ...  
FOXO4        215
NIT1         191
ZBTB10       160
HES7         126
PLK4         113
Name: perturbation_name, Length: 106, dtype: int64

In [7]:
# def sum_to_df(abs_jac):
#     """Takes a 
#     """
#     means = pd.DataFrame(index=adata.var_names, columns=pert_names)

#     for pert_int, pert in enumerate(pert_names):
#         pert_indexing = (adata.obs.perturbation_name.values == pert).reshape(-1)
#         means.iloc[:, pert_int] = abs_jac[pert_indexing].mean(0).numpy()

#     return means

In [8]:
# avg_abs_exp_jac = None
# avg_exp_jac = None
# batch_size = 200  # small bc with 19k genes we run into memory errors

In [44]:
from scvi import REGISTRY_KEYS
pert_names = model.adata_manager.get_state_registry(REGISTRY_KEYS.BATCH_KEY)['categorical_mapping']

In [ ]:
%%time
# sum abs values

batch_size = 50
all_abs_exp_jac = []
for pert in pert_names:  # loop because of memory

    indices = np.where(adata.obs["perturbation_name"] == pert)[0]
    cutoff = len(indices)%batch_size
    if cutoff == 0:
        pass
    else:
        indices = indices[:-cutoff]
    
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
    dl_prime = model._make_data_loader(
        adata=adata, shuffle=True,
        indices=np.random.choice(range(adata.obs.shape[0]), size=len(indices), replace=False),
        batch_size=batch_size
    )
    
    exp_jac_ds = run_expected_jacobian_scanvi(forw, dl_base, dl_prime, apply_abs=True, sum_obs=True)
    
    all_abs_exp_jac.append(exp_jac_ds)

In [69]:
means = pd.DataFrame(torch.stack(all_abs_exp_jac).sum(2).numpy().T, index=adata.var_names, columns=pert_names)

Take the average contribution per gene per **perturbation** to get a ranking of all genes per perturbation.

In [71]:
means.to_csv("../../outputs/expected_gradients/task_1_absolute_expected_grads.csv")

Going to discard this for now because I'd argue it's always less meaningful than the absolute.

In [9]:
# # just sum
# for ds in adata.obs["dataset"].unique():
#     print(ds)
    
#     indices = np.where(adata.obs["dataset"] == ds)[0]
#     batch_size = min(batch_size, int(len(indices) / 11))
#     print(batch_size)
    
#     dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
#     dl_prime = model._make_data_loader(adata=adata, shuffle=True, indices=indices, batch_size=batch_size)
    
#     exp_jac_ds = run_expected_jacobian_scanvi(model.module.classify, dl_base, dl_prime, apply_abs=False, sum_obs=True)
#     if avg_exp_jac is None:
#         avg_exp_jac = exp_jac_ds
#     else:
#         avg_exp_jac += exp_jac_ds

In [13]:
# avg_abs_integr_jac = None
# avg_integr_jac = None

In [ ]:
%%time
batch_size = 200
all_abs_integr_jac = []
for pert in pert_names:  # loop because of memory

    indices = np.where(adata.obs["perturbation_name"] == pert)[0]

    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
    
    integr_jac_ds = run_integrated_jacobian_scanvi(forw, dl_base, apply_abs=True, sum_obs=True)
    
    all_abs_integr_jac.append(integr_jac_ds)

In [ ]:
means_int = pd.DataFrame(torch.stack(all_abs_integr_jac).sum(2).numpy().T, index=adata.var_names, columns=pert_names)

In [ ]:
means_int.to_csv("../../outputs/expected_gradients/task_1_absolute_integrated_grads.csv")

In [ ]:
# # just sum
# for ds in adata.obs["dataset"].unique():
#     print(ds)
    
#     indices = np.where(adata.obs["dataset"] == ds)[0]
#     dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=2000)
    
#     integr_jac_ds = run_integrated_jacobian_scanvi(model.module.classify, dl_base, apply_abs=False, sum_obs=True)
#     if avg_integr_jac is None:
#         avg_integr_jac = integr_jac_ds
#     else:
#         avg_integr_jac += integr_jac_ds

In [23]:
# from scvi import REGISTRY_KEYS
# labels = model.adata_manager.get_state_registry(REGISTRY_KEYS.BATCH_KEY)['categorical_mapping']

In [ ]:
pd.DataFrame(avg_abs_exp_jac.numpy(), columns=list(range(100)), index=adata.var_names.tolist()).to_csv(f"{path}/expected_gradients/task_1_absolute_sum_expected_grads.csv", index_label="Gene")

In [ ]:
# pd.DataFrame(avg_exp_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_expected_grads.csv", index_label="Gene")

In [27]:
pd.DataFrame(avg_abs_integr_jac.numpy(), columns=list(range(100)), index=adata.var_names.tolist()).to_csv(f"{path}/integrated_gradients/task_1_absolute_sum_integrated_grads.csv", index_label="Gene")

In [ ]:
# pd.DataFrame(avg_integr_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_integrated_grads.csv", index_label="Gene")

## Running just EG for multiple scGen models

Load all models.

In [28]:
import os

models = {}
for file in os.listdir(f'{base_path}/models'):
    if 'scgen' in file and 'v2' not in file:
        print('loading', file)
        models['_'.join(file.split('_')[1:])] = scgen.SCGEN.load(f'{base_path}/models/{file}', adata=adata)

models.keys()

loading scgen_norman19_model1_shuffled
INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model1_
         shuffled/model.pt already downloaded                                                
loading scgen_norman19_model1_random
INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model1_
         random/model.pt already downloaded                                                  
loading scgen_norman19_model0_shuffled
INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model0_
         shuffled/model.pt already downloaded                                                
loading scgen_norman19_model4_shuffled
INFO     File /home/icb/yuge.ji/projects/feature-attribution-sc/models/scgen_norman19_model4_
         shuffled/model.pt already downloaded                                                
loading scgen_norman19_model4_random
INFO     File /home/icb/yuge.ji/projects/feature-attribut

dict_keys(['norman19_model1_shuffled', 'norman19_model1_random', 'norman19_model0_shuffled', 'norman19_model4_shuffled', 'norman19_model4_random', 'norman19_model2_random', 'norman19_model2_shuffled', 'norman19_model3_shuffled', 'norman19_model0_random', 'norman19_model3_random'])

In [29]:
def expected_gradients(model, adata, batch_size=200):
    forw = lambda inpt_dict: model.module.inference(inpt_dict['x'])['z']
    
    avg_abs_exp_jac = None

    batch_size = min(batch_size, int(adata.shape[0] / 11))  # ?

    dl_base = model._make_data_loader(adata=adata, shuffle=False, batch_size=batch_size)
    dl_prime = model._make_data_loader(adata=adata, shuffle=True, batch_size=batch_size)

    exp_jac_ds = run_expected_jacobian_scanvi(forw, dl_base, dl_prime, apply_abs=True, sum_obs=True)

    if avg_abs_exp_jac is None:
        avg_abs_exp_jac = exp_jac_ds
    else:
        avg_abs_exp_jac += exp_jac_ds

    return avg_abs_exp_jac

In [31]:
from tqdm import tqdm

for descript, model in tqdm(models.items()):
    exp_jac = expected_gradients(model, adata)
    pd.DataFrame(
        exp_jac.numpy(),
        columns=list(range(100)),  # bc 100 latent features
        index=adata.var_names.tolist()
    ).to_csv(f"{descript}_task1_abssum_EG.csv", index_label="Gene")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [2:57:01<00:00, 1062.15s/it]
